## Installing, importing, etc.

### installing usual stuff

In [ ]:
!pip install -U spacy-experimental
!pip install https://github.com/explosion/spacy-experimental/releases/download/v0.6.0/en_coreference_web_trf-3.4.0a0-py3-none-any.whl
#egg=en_coreference_web_trf
!spacy download en_core_web_sm

In [ ]:
import os
!pip install pandas
!pip install seaborn

In [ ]:
!pip install -e .

In [ ]:
from collections import defaultdict
from datetime import datetime
import os
import pandas as pd
import random
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# SPACY & COREF IMPORTS
import spacy
import spacy_experimental
nlp = spacy.load("en_core_web_sm")
nlp_coref = spacy.load("en_coreference_web_trf")

nlp_coref.replace_listeners("transformer", "coref", ["model.tok2vec"])
nlp_coref.replace_listeners("transformer", "span_resolver", ["model.tok2vec"])

nlp.add_pipe("coref", source=nlp_coref)
nlp.add_pipe("span_resolver", source=nlp_coref)

In [ ]:
from riveter import Riveter

## Information about different functions

### Frequency Threshold
- the minimum number of times entitites/characters have to interact to be included in the results. Default is 1 --> every pair that interacts even once. 

In [ ]:
example_stories = ["I was just thinking about walking down the street, when a car hit a tree. I had to call my doctor to pick me up. I felt so bad I also called Katie, who came in her car. She saved me.",
                   "My doctor fixed my shoe. I thanked him. Then Susan arrived. Now she is calling the doctor too.",
                   "Mary went to the store. She thanked the doctor. He called her. He replied that it was no problem.",
                   "Jack broke the vase."]
text_ids = [0, 1, 2]

### Sap et al Power and Agency

#### load_sap_lexicon(dimension=' ')
-loads sap et al. verb lexicon
- __dimension__ - select 'power' or 'agency'

In [ ]:
riveter = Riveter()
riveter.load_sap_lexicon('power') 
#when doing for my data - the arguments will be (body, ids) where ids = df['id']
riveter.train(example_stories, text_ids)

#### riveter.get_score_totals(frequency_threshold=1)
- gets the total scores of every entity i.e. it's a numeric value representing the strength/direction og a social dynamic within a given context.
- It returns a dictionary of entities and their total scores.

In [ ]:
riveter.get_score_totals(frequency_threshold=1)

#### riveter.plot_scores(number_of_scores=10, title="Personas by Score", frequency_threshold=0)
- creates a barplot showing the final scores across the dataset
- __number_of_scores__: shows only the top or bottom number of scores (optional, integer)
- __title__ : Plot title (optional, string)

In [ ]:
riveter.plot_scores()

In [ ]:
example_stories[0]

#### riveter.get_scores_for_doc(doc_id, frequency_threshold=0)
- get the final scores for all the entities above some frequency threshold in a single document
- __doc_id__: string/integer - show results for this document
- __frequency_threshold__: Entities must be matched to at least this many verbs to appear in the output.
- returns a nested dictionary of document id, entities, and their total scores

In [ ]:
riveter.get_scores_for_doc(0)

#### riveter.plot_scores_for_doc(doc_id, number_of_scores=10, title="Personas by Score", frequency_threshold=0)
- creates a bar plot showing the final scores for a single document
- __number_of_scores__: integer, optional - only shows the top or bottom number of scores
- __title__: string, optional - plot title

In [ ]:
riveter.plot_scores_for_doc(0)

#### riveter.get_persona_polarity_verb_count_dict()
- Gets all the verbs, their frequencies, and whether they contributed positively or negatively to the final scores for every entity. Computed across the whole dataset.
- returns a nested dictionary of entities, positive/negative contributions, verbs, and counts

__how to interpret results:__ <br>
{persona: {
    polarity (positive/negative): {
        verb+role: count
    }
}} 

where: 
- _Persona_: entity (character/object)
- _Polarity_: positive or negative connotation of the interaction
- _Verb+role_: the verb and the syntactic role the persona played
    - verb_nsubj - the persona is an agent of the positive/negative actions
    - verb_dobj - the persona is the direct object/receiver of said actions.
- _Count_: how many times that verb-role combo occured
  

In [ ]:
riveter.get_persona_polarity_verb_count_dict()

#### riveter.plot_verbs_for_persona(persona, figsize=None, output_path=None)
- Creates a heatmap showing the verb counts for a single persona
- Essentially, a breakdown of the verbs that contributed (positively or negativeley) to a persona's final score
- __persona__: string - the entity whose results will be shown in the plot
- __figsize__: tuple, optional - figure dimensions, eg (2,4)
- __output_path__: string, optional - where to save the plot as a file

In [ ]:
riveter.plot_verbs_for_persona('i', figsize=(2,4))

#### riveter.get_persona_counts()
- Get the total counts for the entities (all entity matches, whether or not they were matched to a lexicon verb).
- returns a dictionary of entities and integer counts

In [ ]:
riveter.get_persona_counts()

#### riveter.count_personas_for_doc(doc_id)
- get the entity counts for a single document in the form of a single dictionary

In [ ]:
example_stories[1]

In [ ]:
riveter.count_personas_for_doc(1)

#### riveter.count_scored_verbs_for_doc(doc_id) 
- get the verb counts (verbs that were matched to the lexicon) for a single document
- dictionary of verbs and integer counts

In [ ]:
riveter.count_scored_verbs_for_doc(0)

#### riveter.count_nsubj_for_doc(doc_id, matched_only = False) _OR_ riveter.count_dobj_for_doc(doc_id, matched_only = False)
- gets the noun subject (nsubj) or direct object (dobj) counts for a document
- __matched_only__: boolean - if true, returns only the subject/direct obejects that were matched to identitfied entities
- returns a dictionary of nsubj/dobj and their counts

In [ ]:
riveter.count_nsubj_for_doc(0, matched_only = False)

In [ ]:
riveter.count_dobj_for_doc(0, matched_only = False)

#### riveter.get_persona_cluster(persona)
- returns all the different mentions (words or phrases) in the text that Riveter thinks refer to the same persona from _neuralcoref_ i.e. coreferencing

In [ ]:
riveter.get_persona_cluster('doctor')

### Rashkin's Lexicon of Connotation Frames 
Rashkin et al. defines a connotation frame F(v) as a collection of typed relations and their polarity assignments.

In [ ]:
example_stories = ["I was just thinking about walking down the street, when my shoelace snapped. I had to call my doctor to pick me up. I felt so bad I also called my friend Katie, who came in her car. She was a lifesaver. My friend Jack is nice.",
                   "My doctor fixed my shoe. I thanked him. Then Susan arrived. Now she is calling the doctor too."]
text_ids = [0, 1]

#### Connotation Frames available via Rashkin: 
- __effect__: whether the event denoted by a predicate is good or bad for the entity
- __state__: the likely mental state of an entity as the result of an event
- __value__: whether an entity is presupposed to be valuable
- __writer_perspective__/__reader_perspective__: the directed sentiment from the writer to an entity or the _predicted_ directed sentiment from reader to an entity
- __agent_theme_perspective__/__theme_agent_perspective__: the directed sentiment between the agent and theme (usually reciprocal and not likely to totally contradict each other). 

Connotation frame polarity can be positive, negative, or neutral. 


#### load_rashkin_lexicon(dimension=' ')
- load Rashkin et al., 2016 verb lexicon
- __dimension__: choose one of the connotation frames above

In [ ]:
riveter = Riveter()
riveter.load_rashkin_lexicon('effect') 
riveter.train(example_stories,
              text_ids)

#### riveter.get_documents_for_verb(target_verb)
- find all the documents matched to the verb
- __target_verb__: string - the verb you'd like to match
- returns a list of matched document IDs, and a list of matched document texts

In [ ]:
riveter.get_documents_for_verb('call')

#### riveter.get_documents_for_persona(target_persona)
- find all the documents matched to the persona
- __target_persona__: string, the persona you want to match
- returns a list of matched doc IDs and a list of matched document texts

In [ ]:
riveter.get_documents_for_persona('doctor')